In [4]:
!python3 -m pip install beautifulsoup4 tqdm lxml pandas unidecode pyyaml
# alternative to selenium
# !python3 -m pip install playwright

In [5]:
import os
import numpy as np
import pandas as pd
from datetime import date 
import time
import pickle
import json
import re
import requests
from urllib.error import HTTPError
import urllib.request
from bs4 import BeautifulSoup
import warnings
from pprint import pprint

warnings.filterwarnings('ignore')

In [6]:
import requests 
cookies = {
    '_ga': 'GA1.1.847399952.1685644170',
    'trc_cookie_storage': 'taboola%2520global%253Auser-id%3D30b2cedf-0375-49f9-8ece-fd80c588bbfa-tuct9fb461d',
    '_ga_0VVD9VP56V': 'GS1.1.1686418081.2.0.1686418242.60.0.0',
}

headers = {
    'authority': 'shoesoflasso.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    # 'cookie': '_ga=GA1.1.847399952.1685644170; trc_cookie_storage=taboola%2520global%253Auser-id%3D30b2cedf-0375-49f9-8ece-fd80c588bbfa-tuct9fb461d; _ga_0VVD9VP56V=GS1.1.1686418081.2.0.1686418242.60.0.0',
    'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
}
url = 'https://shoesoflasso.com/episode-list/'
r = requests.get(url, cookies=cookies, headers=headers) 
soup = BeautifulSoup(r.content) 
# print(soup.prettify()) 
web_links = soup.find_all('a') 
actual_web_links = list(set([web_link['href'] for web_link in web_links]))
actual_web_links

['https://shoesoflasso.com/two-aces-season-1-episode-6/',
 'https://shoesoflasso.com/big-week-season-3-ep-4/',
 'https://shoesoflasso.com/season-1-shoes-in-ted-lasso/',
 'https://shoesoflasso.com/privacy-policy/',
 'https://shoesoflasso.com/midnight-train-to-royston-season-2-ep-11/',
 'https://shoesoflasso.com/biscuits-season-1-episode-2/',
 'https://tv.apple.com/us/show/ted-lasso/umc.cmc.vtoh0mn0xn7t3c643xqonfzy',
 'https://shoesoflasso.com/the-hope-that-kills-you-season-1-episode-10/',
 'https://shoesoflasso.com/the-diamond-dogs-season-1-episode-8/',
 'https://shoesoflasso.com/well-never-have-paris-season-3-ep-8/',
 'https://shoesoflasso.com/',
 'https://shoesoflasso.com/ted-lasso-goodbye-earl-season-2-episode-1/',
 'https://shoesoflasso.com/all-apologies-season-1-episode-9/',
 'https://shoesoflasso.com/tan-lines-new-underwear-season-1-episode-5/',
 'https://shoesoflasso.com/smells-like-mean-spirit-season-3-ep-1/',
 'https://shoesoflasso.com/rainbow-season-2-episode-5/',
 'https://sh

In [7]:
import re

def filter_urls(urls):
  """
  Filters a list of URL strings using regex such that each link must end with
  "season" followed by "-" followed by a number followed by "-" followed by
  "episode" followed by "-" followed by a number followed by "/".

  Args:
    urls: A list of URL strings.

  Returns:
    A list of filtered URL strings.
  """

  # Create a regular expression to match the desired pattern.
  pattern1 = re.compile(r'.*season-\d+-ep-\d+\/')
  pattern2 = re.compile(r'.*season-\d+-episode-\d+\/')

  # Filter the list of URL strings.
  filtered_urls = [url for url in urls if re.match(pattern1, url) or re.match(pattern2, url)]

  return filtered_urls

episodes_w_sneaks = filter_urls(actual_web_links)
print(f"Found {len(episodes_w_sneaks)} Ted Lasso episodes with sneakers!")
print(episodes_w_sneaks)

Found 29 Ted Lasso episodes with sneakers!
['https://shoesoflasso.com/two-aces-season-1-episode-6/', 'https://shoesoflasso.com/big-week-season-3-ep-4/', 'https://shoesoflasso.com/midnight-train-to-royston-season-2-ep-11/', 'https://shoesoflasso.com/biscuits-season-1-episode-2/', 'https://shoesoflasso.com/the-hope-that-kills-you-season-1-episode-10/', 'https://shoesoflasso.com/the-diamond-dogs-season-1-episode-8/', 'https://shoesoflasso.com/well-never-have-paris-season-3-ep-8/', 'https://shoesoflasso.com/ted-lasso-goodbye-earl-season-2-episode-1/', 'https://shoesoflasso.com/all-apologies-season-1-episode-9/', 'https://shoesoflasso.com/tan-lines-new-underwear-season-1-episode-5/', 'https://shoesoflasso.com/smells-like-mean-spirit-season-3-ep-1/', 'https://shoesoflasso.com/rainbow-season-2-episode-5/', 'https://shoesoflasso.com/beard-after-hours-season-2-episode-9/', 'https://shoesoflasso.com/pilot-season-1-episode-1/', 'https://shoesoflasso.com/inverting-the-pyramid-of-success-season-2-e

In [24]:
def clean_shoe_name(s):
    if s is None:
        shoe_name = ''
    else:
        shoe_name = s.getText().replace('The Shoe: ', '').replace('The Shoe, ', '').replace('The Show, ', '').replace('The Shoes: ', '')
        shoe_name = shoe_name.strip()
        # manual fix to correct for website error for Nike Tanjun
        shoe_name = shoe_name.replace('of Nate', 'in Midnight Navy/Game Royal/White')
        if shoe_name == 'The Shoes:':
            shoe_name = ''
    return shoe_name

def parse_name_from_post(url):
    global cookies, headers
    r = requests.get(url, cookies=cookies, headers=headers) 
    soup = BeautifulSoup(r.content)
    shoe_heading = soup.find('h2', class_='wp-block-heading')
    try:
        shoe_thumbnail = shoe_heading.find_next('img')['src']
        if 'in-Ted-Lasso' in shoe_thumbnail:
            shoe_thumbnail = ''
    except:
        shoe_thumbnail = ''
    cleaned_name = clean_shoe_name(shoe_heading)
    return cleaned_name, shoe_thumbnail


def parse_episode_info(s):
    # Create a regular expression object to match numbers between dashes or "ep".
    regex = re.compile(r"\d+")
    # Find all matches of the regular expression in the string.
    matches = regex.findall(s)
    season, episode = matches[:2]
    return season, episode

def get_sneaker_names(url):
    global cookies, headers
    episode_str = url[url.find('season'):-1]
    season, episode = parse_episode_info(episode_str)
    # print(season, episode) 
    
    shoe_names = []
    r = requests.get(url, cookies=cookies, headers=headers) 
    soup = BeautifulSoup(r.content)
    web_links = soup.find_all('a', class_='eael-grid-post-link') 
    hrefs = [l['href'] for l in web_links]
    for link in hrefs:
        try:
            shoe_name, shoe_thumbnail = parse_name_from_post(link)
            shoe_names.append([episode_str, season, episode, shoe_name, shoe_thumbnail])
        except:
            print(f"Error with {link}")
    return shoe_names

data = []
for url in episodes_w_sneaks:
    results = get_sneaker_names(url)
    for result in results:
        data.append(result)

In [29]:
import pandas as pd
df = pd.DataFrame(data, columns=['Episode String', 'Season', 'Episode', 'Shoe Name', 'Shoe Thumbnail'])
df = df.drop_duplicates(keep='first')
df = df[df['Shoe Name'] != '']
df['Season'] = df['Season'].astype(int)
df['Episode'] = df['Episode'].astype(int)
df.sort_values(by=['Season', 'Episode'], inplace=True)
df = df.reset_index(drop=True)
df['id'] = pd.Series(df.index).apply(lambda x: f'shoe_{x}')
df

,Episode String,Season,Episode,Shoe Name,Shoe Thumbnail,id
0,season-1-episode-1,1,1,Nike Air Pegasus 92 Lite,,shoe_0
1,season-1-episode-1,1,1,Nike Tanjun,,shoe_1
2,season-1-episode-2,1,2,Jordan Zoom Zero Gravity,https://shoesoflasso.com/wp-content/uploads/20...,shoe_2
3,season-1-episode-2,1,2,Jordan React Havoc Paris Saint-Germain (PSG),https://shoesoflasso.com/wp-content/uploads/20...,shoe_3
4,season-1-episode-3,1,3,Nike Tanjun in Midnight Navy/Game Royal/White,https://shoesoflasso.com/wp-content/uploads/20...,shoe_4
5,season-1-episode-5,1,5,Nike Air Huarache Premium ‘Varsity Jacket’ in Red,https://shoesoflasso.com/wp-content/uploads/20...,shoe_5
6,season-1-episode-6,1,6,Nike Air Zoom Pegasus Lite Total Orange 92,,shoe_6
7,season-1-episode-6,1,6,Odyssey React 2 Flyknit ‘Black Racer Blue’,https://shoesoflasso.com/wp-content/uploads/20...,shoe_7
8,season-1-episode-7,1,7,Nike Air Huarache Premium ‘Varsity Jacket’ in Red,https://shoesoflasso.com/wp-content/uploads/20...,shoe_8
9,season-1-episode-8,1,8,Eric Koston Air Jordan 1 Low ‘Powder Blue’,https://shoesoflasso.com/wp-content/uploads/20...,shoe_9


In [26]:
print('\n'.join(df.iloc[:,-1].to_list()))



https://shoesoflasso.com/wp-content/uploads/2021/10/Jordan-Zoom-Zero-Gravity-Grey-Fog-Obsidian.png
https://shoesoflasso.com/wp-content/uploads/2021/10/Paris-Saint-Germain-x-Jordan-React-Havoc.png
https://shoesoflasso.com/wp-content/uploads/2021/10/tanjun-mens-shoe-profile-1-1024x483.jpeg
https://shoesoflasso.com/wp-content/uploads/2021/10/18-10-2018_nike_airhuaracherunpremium_redsailblue_purple_704830-602_mg_7-copy.png

https://shoesoflasso.com/wp-content/uploads/2021/11/AH1015_011.png.png
https://shoesoflasso.com/wp-content/uploads/2021/10/18-10-2018_nike_airhuaracherunpremium_redsailblue_purple_704830-602_mg_7-copy.png
https://shoesoflasso.com/wp-content/uploads/2021/10/Eric-Conston-Air-Jordan-1-low-1024x454.jpeg
https://shoesoflasso.com/wp-content/uploads/2021/10/Eric-Conston-Air-Jordan-1-low-1024x454.jpeg
https://shoesoflasso.com/wp-content/uploads/2023/06/Screen-Shot-2023-06-05-at-6.21.58-PM-1024x591.png
https://shoesoflasso.com/wp-content/uploads/2023/05/DN6995_101.png-1024x418

In [30]:
df.to_csv('./data/lasso-sneaks.csv', index=False)